In [5]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
 
from bs4 import BeautifulSoup as bs
import time

import requests


https://allofyourbases.com/2018/01/16/mining-twitter-with-selenium/

In [32]:
def init_driver():

    # initiate the driver:
    driver = webdriver.Firefox(
        executable_path=r'geckodriver-v0.24.0-linux64/geckodriver')

    # set a default wait time for the browser [5 seconds here]:
    driver.wait = WebDriverWait(driver, 5)

    return driver


def close_driver(driver):

    driver.close()

    return


def login_twitter(driver, username, password):

    # open the web page in the browser:
    driver.get("https://twitter.com/login")

    # find the boxes for username and password
    username_field = driver.find_element_by_class_name("js-username-field")
    password_field = driver.find_element_by_class_name("js-password-field")

    # enter your username:
    username_field.send_keys(username)
    driver.implicitly_wait(1)

    # enter your password:
    password_field.send_keys(password)
    driver.implicitly_wait(1)

    # click the "Log In" button:
    driver.find_element_by_class_name("EdgeButtom--medium").click()

    return


class wait_for_more_than_n_elements_to_be_present(object):
    def __init__(self, locator, count):
        self.locator = locator
        self.count = count

    def __call__(self, driver):
        try:
            elements = EC._find_elements(driver, self.locator)
            return len(elements) > self.count
        except StaleElementReferenceException:
            return False


def search_twitter(driver, query):

    # wait until the search box has loaded:
    box = driver.wait.until(EC.presence_of_element_located((By.NAME, "q")))

    # find the search box in the html:
    driver.find_element_by_name("q").clear()

    # enter your search string in the search box:
    box.send_keys(query)

    # submit the query (like hitting return):
    box.submit()

    # initial wait for the search results to load
    wait = WebDriverWait(driver, 10)

    try:
        # wait until the first search result is found. Search results will be tweets, which are html list items and have the class='data-item-id':
        wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "li[data-item-id]")))

        # scroll down to the last tweet until there are no more tweets:
        while True:

            # extract all the tweets:
            tweets = driver.find_elements_by_css_selector("li[data-item-id]")

            # find number of visible tweets:
            number_of_tweets = len(tweets)

            # keep scrolling:
            driver.execute_script("arguments[0].scrollIntoView();", tweets[-1])

            try:
                # wait for more tweets to be visible:
                wait.until(wait_for_more_than_n_elements_to_be_present(
                    (By.CSS_SELECTOR, "li[data-item-id]"), number_of_tweets))

            except TimeoutException:
                # if no more are visible the "wait.until" call will timeout. Catch the exception and exit the while loop:
                break

        # extract the html for the whole lot:
        page_source = driver.page_source

    except TimeoutException:

        # if there are no search results then the "wait.until" call in the first "try" statement will never happen and it will time out. So we catch that exception and return no html.
        page_source = None

    return page_source


def extract_tweets(page_source):

    soup = bs(page_source, 'lxml')

    tweets = []
    for li in soup.find_all("li", class_='js-stream-item'):

        # If our li doesn't have a tweet-id, we skip it as it's not going to be a tweet.
        if 'data-item-id' not in li.attrs:
            continue

        else:
            tweet = {
                'tweet_id': li['data-item-id'],
                'text': None,
                'user_id': None,
                'user_screen_name': None,
                'user_name': None,
                'retweets': 0,
                'likes': 0,
                'replies': 0,
                'timestamp': 0,
                'date': ''
            }

            # Tweet Text
            text_p = li.find("p", class_="tweet-text")
            if text_p is not None:
                tweet['text'] = text_p.get_text()

            # Tweet User ID, User Screen Name, User Name
            user_details_div = li.find("div", class_="tweet")
            if user_details_div is not None:
                tweet['user_id'] = user_details_div['data-user-id']
                tweet['user_screen_name'] = user_details_div['data-screen-name']
                tweet['user_name'] = user_details_div['data-name']

            # Tweet Retweets
            retweet_span = li.select(
                "span.ProfileTweet-action--retweet > span.ProfileTweet-actionCount")
            if retweet_span is not None and len(retweet_span) > 0:
                tweet['retweets'] = int(
                    retweet_span[0]['data-tweet-stat-count'])

            # Tweet Likes
            like_span = li.select(
                "span.ProfileTweet-action--favorite > span.ProfileTweet-actionCount")
            if like_span is not None and len(like_span) > 0:
                tweet['likes'] = int(like_span[0]['data-tweet-stat-count'])

            # Tweet Replies
            reply_span = li.select(
                "span.ProfileTweet-action--reply > span.ProfileTweet-actionCount")
            if reply_span is not None and len(reply_span) > 0:
                tweet['replies'] = int(reply_span[0]['data-tweet-stat-count'])
                
            date_span = li.find("span", class_="js-short-timestamp")
            if date_span is not None:
                tweet['timestamp'] = date_span['data-time']
                tweet['date'] = date_span.get_text()
            
            tweets.append(tweet)

    return tweets

In [33]:
driver = init_driver()

login_twitter(driver, 'dybrian008@gmail.com', 'Imaginebreaker12')

hmtl = search_twitter(driver, 'millionpeoplemarch')

tweets = extract_tweets(hmtl)

close_driver(driver)

In [37]:
sub_tweets = tweets[2:4]

In [38]:
sub_tweets

[{'date': '4 Oct 2013',
  'likes': 1,
  'replies': 2,
  'retweets': 1,
  'text': 'A leader vacuum in #MillionPeopleMarch makes it prone to hijack. #justsaying #ScrapPork #Noynoying',
  'timestamp': '1380900185',
  'tweet_id': '386149481978155008',
  'user_id': '341955494',
  'user_name': 'J Ryan Fibs',
  'user_screen_name': 'jrtibster'},
 {'date': '30 Oct 2013',
  'likes': 3,
  'replies': 5,
  'retweets': 5,
  'text': "I tweeted it. It's NOT always about HIM “@peachybrets:  He didn't get it. #MillionPeopleMarch #ScrapPork aren't against him but the system!”",
  'timestamp': '1383136841',
  'tweet_id': '395530695608504320',
  'user_id': '74743293',
  'user_name': 'Jane Uymatiao',
  'user_screen_name': 'philippinebeat'}]

In [39]:
def scrape_followers(driver):
    # initial wait for the search results to load
    wait = WebDriverWait(driver, 30)

    try:
        # wait until the first search result is found. Search results will be tweets, which are html list items and have the class='data-item-id':
        wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "a[data-user-id]")))

        # scroll down to the last tweet until there are no more tweets:
        while True:

            # extract all the tweets:
            followers = driver.find_elements_by_css_selector("div[data-user-id]")

            # find number of visible tweets:
            number_of_followers = len(followers)

            # keep scrolling:
            driver.execute_script("arguments[0].scrollIntoView();", followers[-1])

            try:
                # wait for more tweets to be visible:
                wait.until(wait_for_more_than_n_elements_to_be_present(
                    (By.CSS_SELECTOR, "div[data-user-id]"), number_of_followers))

            except TimeoutException:
                # if no more are visible the "wait.until" call will timeout. Catch the exception and exit the while loop:
                break

        # extract the html for the whole lot:
        page_source = driver.page_source

    except TimeoutException:

        # if there are no search results then the "wait.until" call in the first "try" statement will never happen and it will time out. So we catch that exception and return no html.
        page_source = None

    return page_source

    
def extract_followers(page_source):

    soup = bs(page_source, 'lxml')

    followers = []

    for div in soup.find_all("a", class_='fullname'):
        follower = {
            'screen_name': div['href'][1:],
            'full_name': div.get_text().strip()
        }

        followers.append(follower)

    return followers[1:]

In [40]:
driver = init_driver()

login_twitter(driver, 'dybrian008@gmail.com', 'Imaginebreaker12')

user_followers = {}
for tweet in sub_tweets:
    # open the web page in the browser:
    driver.get("https://twitter.com/" +
               tweet['user_screen_name'] + '/followers')
    followers_html = scrape_followers(driver)
    followers = extract_followers(followers_html)
    user_followers[tweet['user_screen_name']] = followers

close_driver(driver)

In [44]:
user_followers.keys()

dict_keys(['jrtibster', 'philippinebeat'])

In [45]:
user_followers

{'jrtibster': [{'full_name': 'Grab Philippines', 'screen_name': 'grabph'},
  {'full_name': 'BPI', 'screen_name': 'TalktoBPI'},
  {'full_name': 'Burt Steingraeber', 'screen_name': 'BSteingraeber'},
  {'full_name': 'Lea Abogado', 'screen_name': 'leighyaness'},
  {'full_name': 'Lakota', 'screen_name': 'ItSmellsBad'},
  {'full_name': 'red', 'screen_name': 'redhoudini'},
  {'full_name': 'Randy Benosa', 'screen_name': 'Pandy_cocco'},
  {'full_name': 'Patawa Posts', 'screen_name': 'aygrabesyaoh1'},
  {'full_name': 'please follow back', 'screen_name': 'oladejiolabodhe'},
  {'full_name': 'Mc Done', 'screen_name': 'mcdzlove'},
  {'full_name': 'PH Online Voice 2016', 'screen_name': 'onlinevoice2016'},
  {'full_name': 'Save The Soup', 'screen_name': 'LetsSaveTheSoup'},
  {'full_name': 'Super Pop Balloons', 'screen_name': 'SuperPopThe'},
  {'full_name': 'X Wade', 'screen_name': 'georgiymo7jq'},
  {'full_name': 'Juan Dela Cruz', 'screen_name': 'jdcruzph'},
  {'full_name': 'maxine f villafane', 'scre

In [46]:
driver = init_driver()

login_twitter(driver, 'dybrian008@gmail.com', 'Imaginebreaker12')

user_following = {}
for tweet in sub_tweets:
    # open the web page in the browser:
    driver.get("https://twitter.com/" +
               tweet['user_screen_name'] + '/following')
    following_html = scrape_followers(driver)
    following = extract_followers(following_html)
    user_following[tweet['user_screen_name']] = following

close_driver(driver)

In [48]:
user_following.keys()

dict_keys(['jrtibster', 'philippinebeat'])

In [49]:
user_following

{'jrtibster': [{'full_name': 'BPI', 'screen_name': 'TalktoBPI'},
  {'full_name': 'Official Gazette PH', 'screen_name': 'govph'},
  {'full_name': 'Mellow 94.7', 'screen_name': 'mellow947'},
  {'full_name': 'BPI Trade', 'screen_name': 'BPItrade'},
  {'full_name': 'Raquel Fortun', 'screen_name': 'Doc4Dead'},
  {'full_name': 'Tao Essentials', 'screen_name': 'TaoEssentials'},
  {'full_name': '~ஜ۩۞۩ஜ~Senyor~ஜ۩۞۩ஜ~', 'screen_name': 'senyores'},
  {'full_name': 'TradersApprentice', 'screen_name': 'Tony88981'},
  {'full_name': 'ANC On the Money', 'screen_name': 'ANC_OnTheMoney'},
  {'full_name': 'Warren de Guzman', 'screen_name': 'wddeguzman'},
  {'full_name': 'Phil. Stock Exchange', 'screen_name': 'PhStockExchange'},
  {'full_name': 'Official MMDA', 'screen_name': 'MMDA'},
  {'full_name': 'PAGASA-DOST', 'screen_name': 'dost_pagasa'},
  {'full_name': 'Dean Bocobo', 'screen_name': 'deanjorgebocobo'},
  {'full_name': 'Not Will Ferrell', 'screen_name': 'itsWillyFerrell'},
  {'full_name': 'Clarence

In [54]:
import json

with open('tweets.json', 'w') as outfile:
    json.dump(tweets, outfile)
    
with open('user_followers.json', 'w') as outfile:
    json.dump(user_followers, outfile)
    
with open('user_following.json', 'w') as outfile:
    json.dump(user_following, outfile)

In [55]:
with open('tweets.json', 'r') as infile:
    tweets_from_file = json.load(infile)

with open('user_followers.json', 'r') as infile:
    user_followers_from_file = json.load(infile)
    
with open('user_following.json', 'r') as infile:
    user_following_from_file = json.load(infile)

In [63]:
tweets_from_file

[{'date': '4 Oct 2013',
  'likes': 3,
  'replies': 3,
  'retweets': 7,
  'text': 'VIDEO: Thousands join #MillionPeopleMarch http://j.mp/19mK7uf\xa0',
  'timestamp': '1380894458',
  'tweet_id': '386125462285471744',
  'user_id': '44728980',
  'user_name': 'ABS-CBN News Channel',
  'user_screen_name': 'ANCALERTS'},
 {'date': '4 Oct 2013',
  'likes': 3,
  'replies': 2,
  'retweets': 4,
  'text': 'PHOTO: Guest artists perform at #MillionPeopleMarch in Makati pic.twitter.com/smsBljp1o8',
  'timestamp': '1380888906',
  'tweet_id': '386102174230388736',
  'user_id': '44728980',
  'user_name': 'ABS-CBN News Channel',
  'user_screen_name': 'ANCALERTS'},
 {'date': '4 Oct 2013',
  'likes': 1,
  'replies': 2,
  'retweets': 1,
  'text': 'A leader vacuum in #MillionPeopleMarch makes it prone to hijack. #justsaying #ScrapPork #Noynoying',
  'timestamp': '1380900185',
  'tweet_id': '386149481978155008',
  'user_id': '341955494',
  'user_name': 'J Ryan Fibs',
  'user_screen_name': 'jrtibster'},
 {'date

In [64]:
user_followers_from_file

{'jrtibster': [{'full_name': 'Grab Philippines', 'screen_name': 'grabph'},
  {'full_name': 'BPI', 'screen_name': 'TalktoBPI'},
  {'full_name': 'Burt Steingraeber', 'screen_name': 'BSteingraeber'},
  {'full_name': 'Lea Abogado', 'screen_name': 'leighyaness'},
  {'full_name': 'Lakota', 'screen_name': 'ItSmellsBad'},
  {'full_name': 'red', 'screen_name': 'redhoudini'},
  {'full_name': 'Randy Benosa', 'screen_name': 'Pandy_cocco'},
  {'full_name': 'Patawa Posts', 'screen_name': 'aygrabesyaoh1'},
  {'full_name': 'please follow back', 'screen_name': 'oladejiolabodhe'},
  {'full_name': 'Mc Done', 'screen_name': 'mcdzlove'},
  {'full_name': 'PH Online Voice 2016', 'screen_name': 'onlinevoice2016'},
  {'full_name': 'Save The Soup', 'screen_name': 'LetsSaveTheSoup'},
  {'full_name': 'Super Pop Balloons', 'screen_name': 'SuperPopThe'},
  {'full_name': 'X Wade', 'screen_name': 'georgiymo7jq'},
  {'full_name': 'Juan Dela Cruz', 'screen_name': 'jdcruzph'},
  {'full_name': 'maxine f villafane', 'scre

In [65]:
user_following_from_file

{'jrtibster': [{'full_name': 'BPI', 'screen_name': 'TalktoBPI'},
  {'full_name': 'Official Gazette PH', 'screen_name': 'govph'},
  {'full_name': 'Mellow 94.7', 'screen_name': 'mellow947'},
  {'full_name': 'BPI Trade', 'screen_name': 'BPItrade'},
  {'full_name': 'Raquel Fortun', 'screen_name': 'Doc4Dead'},
  {'full_name': 'Tao Essentials', 'screen_name': 'TaoEssentials'},
  {'full_name': '~ஜ۩۞۩ஜ~Senyor~ஜ۩۞۩ஜ~', 'screen_name': 'senyores'},
  {'full_name': 'TradersApprentice', 'screen_name': 'Tony88981'},
  {'full_name': 'ANC On the Money', 'screen_name': 'ANC_OnTheMoney'},
  {'full_name': 'Warren de Guzman', 'screen_name': 'wddeguzman'},
  {'full_name': 'Phil. Stock Exchange', 'screen_name': 'PhStockExchange'},
  {'full_name': 'Official MMDA', 'screen_name': 'MMDA'},
  {'full_name': 'PAGASA-DOST', 'screen_name': 'dost_pagasa'},
  {'full_name': 'Dean Bocobo', 'screen_name': 'deanjorgebocobo'},
  {'full_name': 'Not Will Ferrell', 'screen_name': 'itsWillyFerrell'},
  {'full_name': 'Clarence